# Apply CNN on Synthetic Data

## Importing Requirements

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.nn import init
import numpy as np
from torch import nn
from torch.autograd import Variable
import math
import torch
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from box_gen import get_array_with_box_at_pos

In [ ]:
# import data
X = np.load("RCNN/data/sliding_square.npy")
y = np.load("RCNN/data/sliding_square_target.npy")

In [ ]:
# split data
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42)

**Visualizing data**

In [ ]:
# print out x,y pairs
for frame, next_pos in zip(x_train[:5], y_train[:5]):
    ax2 = plt.subplot(221)
    ax2.imshow(frame)
    ax2.set_title('X')
    ax1 = plt.subplot(222)
    ax1.imshow(get_array_with_box_at_pos(next_pos))
    ax1.set_title('Y')
    plt.show()


In [ ]:
print("X Train Shape : ", x_train.shape)

# Build CNN Model
To see what happens when CNN s used for predicting the position of the box given the current position. our CNN model looks like as given below. 
It has one Conv1D layer, followed by dense layers. 

In [ ]:
class CNN(nn.Module):
    def __init__(self ):
        super(CNN, self).__init__()
        self.conv1D = nn.Conv1d(in_channels=5,out_channels=2, kernel_size=3)
        self.dense1 = nn.Linear(in_features=2*48,out_features=10)
        self.dense2 = nn.Linear(in_features=10,out_features=1)
        
    def forward(self, input):
        conv_out = self.conv1D(input)
        conv_out_reshape =  conv_out.view(-1,48*2)
        dense1_out = self.dense1(conv_out_reshape)
        dense2_out = self.dense2(dense1_out)
        return dense2_out

In [ ]:
cnn_model  = CNN()

**Defining loss and optimizer**

In [ ]:
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.1)

# Training 
The input to this network will be A 2D array with the position of the box in the frame. The expected output will be the next position of the box in the frame. 

In [ ]:
epochs = 100
losses  = []
for i in range(epochs):
    x = Variable(torch.Tensor(x_train).type(torch.FloatTensor))
    optimizer.zero_grad()
    predicted = cnn_model(x)
    loss = criteria(predicted, Variable(torch.Tensor(y_train.reshape(-1,1))))
    losses.append(loss.item())
    loss.backward()
    optimizer.step()

## Plotting progress

In [ ]:
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Losses")
plt.title("Decrease in loss as the training progresses")

# Visualizing Predictions

In [ ]:
test_loss = 0
all_predictions = []
all_label = []
for frame, output in zip(x_test, y_test):
    x = Variable(torch.Tensor(frame).unsqueeze(0).type(torch.FloatTensor))
    predicted = cnn_model(x)
    
    ax1 = plt.subplot(331)
    ax1.imshow(x[0])
    ax1.set_title('X')
    plt.yticks([])

    ax2 = plt.subplot(332)
    ax2.imshow(get_array_with_box_at_pos(predicted.detach().numpy()[0][0]))
    ax2.set_title('y_pred'+ " = "+ str(round(predicted.detach().numpy()[0][0],2)))
    plt.yticks([])
    
    ax3 = plt.subplot(333)
    ax3.imshow(get_array_with_box_at_pos(output))
    ax3.set_title('y'+ " = "+ str(round(output,2)))
    plt.yticks([])
    plt.show()
    
    all_predictions.append(predicted.item())
    all_label.append(output)

test_loss += mean_squared_error(all_predictions, all_label)

In [ ]:
print("The final test loss : ", test_loss)